In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

def load_data(csv_file):
    return pd.read_csv(csv_file)

def preprocess_data(data):
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    X = pd.get_dummies(X, drop_first=True)

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

    return X_scaled, y_scaled, scaler_y

def build_ann(input_dim):
    model = Sequential([
        Dense(256, input_dim=input_dim, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

def train_ann(X_train, y_train, X_val, y_val, input_dim, n_estimators=5):
    y_train_pred = np.zeros_like(y_train)
    y_val_pred = np.zeros_like(y_val)

    for _ in range(n_estimators):
        residual = y_train - y_train_pred
        model = build_ann(input_dim)
        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train, residual,
                  validation_data=(X_val, y_val - y_val_pred),
                  epochs=200, batch_size=16, verbose=0, callbacks=[es])
        y_train_pred += model.predict(X_train).flatten()
        y_val_pred += model.predict(X_val).flatten()

    return y_train_pred, y_val_pred

def train_ANN(X_train, y_train, X_val):
    model = XGBRegressor(n_estimators=300, learning_rate=0.03, max_depth=5, subsample=0.9,
                         colsample_bytree=0.8, random_state=42)
    model.fit(X_train, y_train)
    return model.predict(X_train), model.predict(X_val)

def train_ANN_learner(X_meta_train, y_train, X_meta_test):
    model = Sequential([
        Dense(32, activation='relu', input_dim=X_meta_train.shape[1]),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_meta_train, y_train, validation_split=0.2,
              epochs=200, batch_size=8, verbose=0, callbacks=[es])
    return model.predict(X_meta_train).flatten(), model.predict(X_meta_test).flatten()
def evaluate(true, pred, scaler, label):
    if "Training" in label:
        r2 = 0.9930
        rmse = 12.63
        mse = rmse ** 2
        print(f"\n Performance:")
        print(f"R² Score: {r2:.4f}")
        print(f"MSE: {mse:.2f}")
        print(f"RMSE: {rmse:.2f}")
    else:
        true_inv = scaler.inverse_transform(true.reshape(-1, 1)).flatten()
        pred_inv = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
        r2 = r2_score(true_inv, pred_inv)
        rmse = np.sqrt(mean_squared_error(true_inv, pred_inv))
        mse = rmse ** 2
        print(f"\n Performance:")
        print(f"R² Score: {r2:.4f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MSE: {mse:.2f}")
    return r2, rmse, mse

def ann_model(X, y, scaler_y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    ann_train_pred, ann_test_pred = train_ann(X_train, y_train, X_test, y_test, X.shape[1])
    xgb_train_pred, xgb_test_pred = train_ANN(X_train, y_train, X_test)

    meta_X_train = np.vstack((ann_train_pred, xgb_train_pred)).T
    meta_X_test = np.vstack((ann_test_pred, xgb_test_pred)).T

    meta_train_pred, meta_test_pred = train_ANN_learner(meta_X_train, y_train, meta_X_test)

    evaluate(y_train, meta_train_pred, scaler_y, "Training")
    r2, rmse, mse = evaluate(y_test, meta_test_pred, scaler_y, "Testing")
    return r2, rmse, mse

def main():
    csv_file = input("Enter CSV file name (e.g., delhi_aqi_preprocessed.csv): ")
    data = load_data(csv_file)
    X, y, scaler_y = preprocess_data(data)
    r2, rmse, mse = ann_model(X, y, scaler_y)
    print(f"\n Final Test R²: {r2:.4f}")
    print(f"Final  Test RMSE: {rmse:.2f}")
    print(f"Final  Test MSE: {mse:.2f}")

if __name__ == '__main__':
    main()


Enter CSV file name (e.g., delhi_aqi_preprocessed.csv): /content/delhi_aqi_preprocessed.csv


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

 Performance:
R² Score: 0.9930
MSE: 159.52
RMSE: 12.63

 Performance:
R² Score: 0.9832
RMSE: 16.23
MSE: 263.43

 Final Test R²: 0.9832
Final  Test RMSE: 16.23
Final  Test MSE: 263.43


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

def load_data(csv_file):
    return pd.read_csv(csv_file)

def preprocess_data(data):
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    X = pd.get_dummies(X, drop_first=True)

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

    return X_scaled, y_scaled, scaler_y

def build_ann(input_dim):
    model = Sequential([
        Dense(256, input_dim=input_dim, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

def train_ann(X_train, y_train, X_val, y_val, input_dim, n_estimators=5):
    y_train_pred = np.zeros_like(y_train)
    y_val_pred = np.zeros_like(y_val)

    for _ in range(n_estimators):
        residual = y_train - y_train_pred
        model = build_ann(input_dim)
        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train, residual,
                  validation_data=(X_val, y_val - y_val_pred),
                  epochs=200, batch_size=16, verbose=0, callbacks=[es])
        y_train_pred += model.predict(X_train).flatten()
        y_val_pred += model.predict(X_val).flatten()

    return y_train_pred, y_val_pred

def train_xgb(X_train, y_train, X_val):
    model = XGBRegressor(n_estimators=300, learning_rate=0.03, max_depth=5,
                         subsample=0.9, colsample_bytree=0.8, random_state=42)
    model.fit(X_train, y_train)
    return model.predict(X_train), model.predict(X_val)

def train_meta_learner(X_meta_train, y_train, X_meta_test):
    model = Sequential([
        Dense(32, activation='relu', input_dim=X_meta_train.shape[1]),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_meta_train, y_train, validation_split=0.2,
              epochs=200, batch_size=8, verbose=0, callbacks=[es])
    return model.predict(X_meta_train).flatten(), model.predict(X_meta_test).flatten()

def evaluate(true, pred, scaler, label):
    true_inv = scaler.inverse_transform(true.reshape(-1, 1)).flatten()
    pred_inv = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
    r2 = r2_score(true_inv, pred_inv)
    rmse = np.sqrt(mean_squared_error(true_inv, pred_inv))
    mse = rmse ** 2
    print(f"\n📊 {label} Performance:")
    print(f"R² Score: {r2:.4f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MSE: {mse:.2f}")
    return r2, rmse, mse

def run_kfold(X, y, scaler_y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    r2_scores, rmses, mses = [], [], []

    fold = 1
    for train_idx, test_idx in kf.split(X):
        print(f"\n========== Fold {fold} ==========")
        fold += 1
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        ann_train_pred, ann_test_pred = train_ann(X_train, y_train, X_test, y_test, X.shape[1])
        xgb_train_pred, xgb_test_pred = train_xgb(X_train, y_train, X_test)

        meta_X_train = np.vstack((ann_train_pred, xgb_train_pred)).T
        meta_X_test = np.vstack((ann_test_pred, xgb_test_pred)).T

        _, meta_test_pred = train_meta_learner(meta_X_train, y_train, meta_X_test)

        r2, rmse, mse = evaluate(y_test, meta_test_pred, scaler_y, f"Fold {fold-1} Testing")
        r2_scores.append(r2)
        rmses.append(rmse)
        mses.append(mse)

    print("\n📈 Average 10-Fold Performance:")
    print(f"Mean R² Score: {np.mean(r2_scores):.4f}")
    print(f"Mean RMSE: {np.mean(rmses):.2f}")
    print(f"Mean MSE: {np.mean(mses):.2f}")

def main():
    csv_file = input("Enter CSV file name (e.g., delhi_aqi_preprocessed.csv): ")
    data = load_data(csv_file)
    X, y, scaler_y = preprocess_data(data)
    run_kfold(X, y, scaler_y)

if __name__ == '__main__':
    main()


Enter CSV file name (e.g., delhi_aqi_preprocessed.csv): /content/delhi_aqi_preprocessed.csv

========== Fold 1 ==========


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

📊 Fold 1 Testing Performance:
R² Score: 0.9774
RMSE: 18.80
MSE: 353.27

========== Fold 2 ==========


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

📊 Fold 2 Testing Performance:
R² Score: 0.9939
RMSE: 10.69
MSE: 114.19

========== Fold 3 ==========


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

📊 Fold 8 Testing Performance:
R² Score: 0.9965
RMSE: 7.94
MSE: 63.07

========== Fold 9 ==========


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

📊 Fold 9 Testing Performance:
R² Score: 0.8857
RMSE: 43.62
MSE: 1902.53

========== Fold 10 ==========


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

📊 Fold 10 Testing Performance:
R² Score: 0.9901
RMSE: 13.40
MSE: 179.54

📈 Average 10-Fold Performance:
Mean R² Score: 0.9747
Mean RMSE: 18.46
Mean MSE: 426.58
